# Convolutional Neural Network (CNN) – Minh hoạ với **số 0**
Notebook này trình bày lý thuyết và thực hành CNN theo phong cách *CNN‑LAB.ipynb*, nhưng áp dụng cụ thể cho chữ số **0**.

- Phần **lý thuyết**: tích chập, ReLU, pooling, fully‑connected qua ví dụ ma trận $6\times6$ số 0.
- Phần **thực hành**: xây dựng & huấn luyện CNN nhỏ trên bộ MNIST (PyTorch).

## 1. CNN là gì?
CNN (Convolutional Neural Network) giúp máy tính "nhìn" ảnh bằng cách quét cửa sổ nhỏ (kernel) qua ảnh, tìm đặc trưng cục bộ rồi tổng hợp.
Ví dụ: mắt người nhận dạng mèo qua tai tam giác, mắt tròn, ria mép dài … CNN cũng học các đặc trưng tương tự.

Hình dung chữ số *7*: CNN thấy một **nét ngang** và một **nét chéo**, kết luận *7*. Với **số 0** sẽ là khung tròn viền ngoài.

## 2. Các thành phần chính của CNN
1. **Convolution layer** – tìm đặc trưng cục bộ.
2. **Activation (ReLU)** – giữ giá trị dương mạnh, bỏ âm.
3. **Pooling layer** – tóm tắt (giảm kích thước) feature map.
4. **Fully Connected layer** – ghép đặc trưng → phân loại.

### 2.1. Tầng tích chập – ví dụ số 0 (ma trận 6×6)
Ảnh gốc 6×6 (1 = nét, 0 = nền):

In [ ]:

import numpy as np, torch, torch.nn as nn, matplotlib.pyplot as plt
img = np.array([
    [1,1,1,1,1,1],
    [1,0,0,0,0,1],
    [1,0,0,0,0,1],
    [1,0,0,0,0,1],
    [1,0,0,0,0,1],
    [1,1,1,1,1,1],
], dtype=np.float32)
plt.imshow(img, cmap='gray'); plt.title('Ảnh số 0 (6×6)'); plt.axis('off')


**Kernel $3\times3$ tìm cạnh ngang:**
```text
 1  1  1
 0  0  0
-1 -1 -1
```
Thực hiện tích chập (stride 1, no padding) → thu được feature map $4\times4$.

In [ ]:

kernel = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float32)
conv = nn.Conv2d(1,1,3,bias=False)
conv.weight.data[0,0] = torch.tensor(kernel)
with torch.no_grad():
    fmap = conv(torch.tensor(img).unsqueeze(0).unsqueeze(0)).squeeze().numpy()
print("Feature map 4×4:")
print(fmap.astype(int))


Feature map làm nổi bật **đường ngang phía trên** của số 0.

### 2.2. Hàm kích hoạt ReLU

In [ ]:

fmap_relu = np.maximum(fmap, 0)
print("Sau ReLU:")
print(fmap_relu.astype(int))


Giá trị âm (mép dưới) =⇒ 0, chỉ giữ đặc trưng mạnh (dương).

### 2.3. Max Pooling 2×2 (stride 2)

In [ ]:

pool = nn.MaxPool2d(2,2)
with torch.no_grad():
    pooled = pool(torch.tensor(fmap_relu).unsqueeze(0).unsqueeze(0)).squeeze().numpy()
print("Feature map 2×2 sau pooling:")
print(pooled.astype(int))


Kích thước giảm, vẫn giữ hai giá trị 3 đại diện cạnh ngang mạnh.

### 2.4. Fully Connected – phân biệt "0" / "không 0" (2 lớp)
Flatten vector $[3,3,0,0]$, trọng số & bias giả lập để minh hoạ (0.92 xác suất số 0).

In [ ]:

z = torch.tensor(pooled.flatten(), dtype=torch.float32)
W = torch.tensor([[0.2,0.2,0,0],
                  [-0.2,-0.2,0,0]])
b = torch.tensor([0.,0.])
logits = W @ z + b
prob = torch.softmax(logits, dim=0)
print("Logits:", logits.numpy())
print("Softmax probs:", prob.numpy())


## 3. Tóm tắt quy trình
- **Convolution**: tìm cạnh ngang trên → giá trị 3.
- **ReLU**: bỏ giá trị âm, giữ cạnh trên.
- **Pooling**: giảm kích thước, vẫn giữ đặc trưng mạnh.
- **FC**: kết luận ảnh là *số 0* với xác suất ~92 %.

## 4. Ứng dụng thực tế của CNN
- Nhận diện khuôn mặt (Facebook, FaceID).
- Xe tự lái (phát hiện biển báo, người đi bộ).
- Y khoa (phát hiện khối u trong X‑ray, MRI).
- Phân tích video, object detection,…

## 5. Thực hành – Huấn luyện CNN nhỏ trên MNIST

In [ ]:

import torchvision, torchvision.transforms as transforms, torch.optim as optim
transform = transforms.ToTensor()
train_ds = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_ds  = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_ds , batch_size=64, shuffle=False)

class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.pool  = nn.MaxPool2d(2,2)
        self.fc    = nn.Linear(32*5*5,10)
    def forward(self,x):
        x=self.pool(torch.relu(self.conv1(x)))
        x=self.pool(torch.relu(self.conv2(x)))
        x=x.view(-1,32*5*5)
        return self.fc(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MNIST_CNN().to(device)
crit = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

loss_hist, acc_hist = [], []
for epoch in range(5):
    model.train(); run_loss=0; correct=0; total=0
    for imgs,lbls in train_loader:
        imgs,lbls=imgs.to(device),lbls.to(device)
        opt.zero_grad()
        out=model(imgs)
        loss=crit(out,lbls)
        loss.backward(); opt.step()
        run_loss+=loss.item()
        _,pred=torch.max(out,1)
        total+=lbls.size(0); correct+=(pred==lbls).sum().item()
    print(f'Epoch {epoch+1}: loss={run_loss/len(train_loader):.4f}, acc={100*correct/total:.2f}%')


Huấn luyện đạt ~98% accuracy sau 5 epoch. Bạn có thể tiếp tục cell test / visualize từ notebook gốc nếu muốn.